Программно реализовать извлечение стандартных именованных сущностей таких категорий как Person, Location, Organization.
Корпуса с данными:

- [BSNLP 2017 — Balto-Slavic Natural Language Processing](http://bsnlp-2017.cs.helsinki.fi/shared_task_results.html);
- [Ukranian NER annotation project](https://github.com/lang-uk/ner-uk).

Корпус выбирайте любой. Метод решения поставленной задачи тоже любой. Результаты будем обсуждать в пятницу. Удачи!

In [18]:
from nltk import pos_tag
from nltk.tag import PerceptronTagger, UnigramTagger, BigramTagger

#### Read filenames for train and test part

In [19]:
def get_lines(file):
    return [line.replace('\n', '') for line in file.readlines() if line and not line.isspace()]

with open("ner-uk-master/doc/dev-test-split.txt") as file:
    filenames = get_lines(file)
    train_filenames = filenames[1:156]
    test_filenames = filenames[159:231]

#### Read train and test documents
Fixed filenames in dev-test-split.txt

In [20]:
def get_tags_and_words(line):
    return (line.split("\t")[2], line.split("\t")[1].split(' ')[0])

def read_file(filename):
    with open(filename, encoding='utf-8') as file:
        return [get_tags_and_words(line) for line in get_lines(file)]

def read_data(filenames):
    return [read_file("ner-uk-master/data/" + filename + '.ann') for filename in filenames]

def remove_falsy(lst):
    return [x for x in lst if x]
    
training_corpus = remove_falsy(read_data(train_filenames))
test_corpus = remove_falsy(read_data(test_filenames))

Use the pretrain model (the default constructor)? Or just train?

In [21]:
with open("ner-uk-master/doc/sent-tokenization.txt", encoding="utf-8") as file:
    sentences = get_lines(file)   

In [22]:
unitagger = UnigramTagger(training_corpus)

In [23]:
unitagger.evaluate(test_corpus)

0.21092029942756496

In [24]:
bitagger = BigramTagger(training_corpus, backoff=unitagger)

In [25]:
bitagger.evaluate(test_corpus)

0.21092029942756496

In [26]:
tagger = PerceptronTagger(load=False)
tagger.train(training_corpus)

In [27]:
tagger.evaluate(test_corpus)

0.6961690885072656

In [28]:
import re

word_regex = r'\w+://(?:[a-zA-Z]|[0-9]|[$-_@.&+])+' + \
 '|(?:ім.|о.|вул.|просп.|буд.|пров.|пл.|г.|р.|див.|п.|с.|м.|кв.|акад.|доц.|проф.)\s(?:\S+)' + \
 '|[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+' + \
 '|[0-9]+-[а-яА-ЯіїІЇ\'’`]+' + \
 '|[+-]?[0-9](?:[0-9,.-]*[0-9])?' + \
 '|[\w](?:[\w\'’`-]?[\w]+)*' + \
 '|\w.(?:\w.)+\w?'

def word_tokenize_regex(sentence):
    return re.findall(word_regex, sentence)

for sent in sentences:
    if sent and not sent.isspace():
        words = word_tokenize_regex(sent)
        pos_tags_words = tagger.tag(words)
        print(pos_tags_words)

[('Це', 'ОРГ'), ('просте', 'ЛОК'), ('речення', 'ОРГ')]
[('Панк-рок', 'ПЕРС'), ('напрям', 'ЛОК'), ('у', 'ПЕРС'), ('рок-музиці', 'ОРГ'), ('що', 'ПЕРС'), ('виник', 'РІЗН'), ('у', 'ОРГ'), ('середині', 'ЛОК'), ('1970-х', 'ОРГ'), ('рр', 'ЛОК'), ('у', 'ЛОК'), ('США', 'ЛОК'), ('і', 'ЛОК'), ('Великобританії', 'ЛОК')]
[('Разом', 'ПЕРС'), ('із', 'ПЕРС'), ('втечами', 'ЛОК'), ('вже', 'ЛОК'), ('у', 'ЛОК'), ('XV', 'ЛОК'), ('ст', 'ЛОК'), ('почастішали', 'ЛОК'), ('збройні', 'ЛОК'), ('виступи', 'ЛОК'), ('селян', 'ЛОК')]
[('На', 'ПЕРС'), ('початок', 'ЛОК'), ('1994', 'ОРГ'), ('р. державний', 'ОРГ'), ('борг', 'ОРГ'), ('України', 'ЛОК'), ('становив', 'ЛОК'), ('4,8', 'ЛОК'), ('млрд', 'ЛОК'), ('дол', 'ЛОК')]
[('Київ', 'ЛОК'), ('вул. Сагайдачного,', 'ЛОК'), ('буд. 43,', 'ЛОК'), ('кв. 4.', 'ЛОК')]
[('Наша', 'ПЕРС'), ('зустріч', 'ЛОК'), ('з', 'ЛОК'), ('А', 'ЛОК'), ('Марчуком', 'ПЕРС'), ('і', 'ПЕРС'), ('Г', 'ПЕРС'), ('В', 'ПЕРС'), ('Тріскою', 'ПЕРС'), ('відбулася', 'ЛОК'), ('в', 'ЛОК'), ('грудні', 'ЛОК'), ('минул

In [29]:
with open("ner-uk-master/doc/word-tokenization.txt", encoding="utf-8") as file:
    small_sentences = get_lines(file)

In [30]:
for sent in small_sentences:
    if sent and not sent.isspace():
        words = word_tokenize_regex(sent)
        pos_tags_words = tagger.tag(words)
        print(pos_tags_words)

[('http://youtube.com:80/herewego?start=11&quality=high%3F', 'ПЕРС')]
[('http://youtube.com:80/herewego?start=11&quality=high%3F', 'ПЕРС')]
[('300', 'ПЕРС'), ('грн', 'РІЗН'), ('на', 'ПЕРС'), ('балансі', 'ПЕРС')]
[('300', 'ПЕРС'), ('грн', 'РІЗН'), ('на', 'ПЕРС'), ('балансі', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('2,2', 'РІЗН'), ('мільйона', 'РІЗН')]
[('надійшло', 'ПЕРС'), ('2,2', 'РІЗН'), ('мільйона', 'РІЗН')]
[('надійшло', 'ПЕРС'), ('84,46', 'РІЗН'), ('мільйона', 'РІЗН')]
[('надійшло', 'ПЕРС'), ('84,46', 'РІЗН'), ('мільйона', 'РІЗН')]
[('в', 'ЛОК'), ('1996,1997,1998', 'ОРГ')]
[('в', 'ЛОК'), ('1996', 'ОРГ'), ('1997', 'ОРГ'), ('1998', 'ОРГ')]
[('надійшло', 'ПЕРС'), ('2', 'РІЗН'), ('000', 'РІЗН'), ('тон', 'РІЗН')]
[('надійшло', 'ПЕРС'), ('2', 'РІЗН'), ('000', 'РІЗН'), ('тон', 'РІЗН')]
[('сталося', 'ПЕРС'), ('14.07.2001', 'РІЗН'), ('вночі', 'РІЗН')]
[('сталося', 'ПЕРС'), ('14.07.2001', 'РІЗН'), ('вночі', 'РІЗН')]
[('вчора', 'ЛОК'), ('о', 'ПЕРС'), ('7.30', 'РІЗН'), ('ранку', 'РІЗН')]
[('вчора', 

In [31]:
import os

filenames = [filename[:-4] for filename in os.listdir('ner-uk-master/data/') \
             if filename.endswith(".ann") and not filename.endswith(".tok.ann")]

tagged_sentences = remove_falsy(read_data(filenames))

idx = int(len(tagged_sentences)*0.3)
train_sentences = tagged_sentences[idx:]
test_sentences = tagged_sentences[:idx]

tagger = PerceptronTagger(load=False)
tagger.train(train_sentences)

In [32]:
tagger.evaluate(test_sentences) 

0.5276211950394588

In [33]:
for sent in sentences:
    if sent and not sent.isspace():
        words = word_tokenize_regex(sent)
        pos_tags_words = tagger.tag(words)
        print(pos_tags_words)      

[('Це', 'ПЕРС'), ('просте', 'ЛОК'), ('речення', 'ПЕРС')]
[('Панк-рок', 'ПЕРС'), ('напрям', 'ЛОК'), ('у', 'ПЕРС'), ('рок-музиці', 'ЛОК'), ('що', 'ПЕРС'), ('виник', 'ЛОК'), ('у', 'ЛОК'), ('середині', 'ЛОК'), ('1970-х', 'ОРГ'), ('рр', 'ОРГ'), ('у', 'ОРГ'), ('США', 'ЛОК'), ('і', 'ЛОК'), ('Великобританії', 'ЛОК')]
[('Разом', 'ПЕРС'), ('із', 'ПЕРС'), ('втечами', 'ЛОК'), ('вже', 'ЛОК'), ('у', 'ЛОК'), ('XV', 'ЛОК'), ('ст', 'ЛОК'), ('почастішали', 'ЛОК'), ('збройні', 'ЛОК'), ('виступи', 'ЛОК'), ('селян', 'ЛОК')]
[('На', 'ПЕРС'), ('початок', 'ЛОК'), ('1994', 'ЛОК'), ('р. державний', 'ПЕРС'), ('борг', 'ПЕРС'), ('України', 'ЛОК'), ('становив', 'ПЕРС'), ('4,8', 'РІЗН'), ('млрд', 'ПЕРС'), ('дол', 'ПЕРС')]
[('Київ', 'ЛОК'), ('вул. Сагайдачного,', 'ЛОК'), ('буд. 43,', 'ЛОК'), ('кв. 4.', 'ЛОК')]
[('Наша', 'ПЕРС'), ('зустріч', 'ПЕРС'), ('з', 'ПЕРС'), ('А', 'ПЕРС'), ('Марчуком', 'ПЕРС'), ('і', 'ПЕРС'), ('Г', 'ПЕРС'), ('В', 'ПЕРС'), ('Тріскою', 'ПЕРС'), ('відбулася', 'ЛОК'), ('в', 'ЛОК'), ('грудні', 'ЛОК'

In [34]:
for sent in small_sentences:
    if sent and not sent.isspace():
        words = word_tokenize_regex(sent)
        pos_tags_words = tagger.tag(words)
        print(pos_tags_words) 

[('http://youtube.com:80/herewego?start=11&quality=high%3F', 'ПЕРС')]
[('http://youtube.com:80/herewego?start=11&quality=high%3F', 'ПЕРС')]
[('300', 'ПЕРС'), ('грн', 'ПЕРС'), ('на', 'ПЕРС'), ('балансі', 'ПЕРС')]
[('300', 'ПЕРС'), ('грн', 'ПЕРС'), ('на', 'ПЕРС'), ('балансі', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('2,2', 'РІЗН'), ('мільйона', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('2,2', 'РІЗН'), ('мільйона', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('84,46', 'РІЗН'), ('мільйона', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('84,46', 'РІЗН'), ('мільйона', 'ПЕРС')]
[('в', 'ПЕРС'), ('1996,1997,1998', 'РІЗН')]
[('в', 'ЛОК'), ('1996', 'РІЗН'), ('1997', 'ПЕРС'), ('1998', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('2', 'РІЗН'), ('000', 'ПЕРС'), ('тон', 'ПЕРС')]
[('надійшло', 'ПЕРС'), ('2', 'РІЗН'), ('000', 'ПЕРС'), ('тон', 'ПЕРС')]
[('сталося', 'ПЕРС'), ('14.07.2001', 'РІЗН'), ('вночі', 'ЛОК')]
[('сталося', 'ПЕРС'), ('14.07.2001', 'РІЗН'), ('вночі', 'ЛОК')]
[('вчора', 'ЛОК'), ('о', 'ПЕРС'), ('7.30', 'РІЗН'), ('ранку', 'ПЕРС')]
[('вчора